In [1]:
from methods import *
import os, shutil

In [7]:
#user inputs
# TODO: add EDA4

#load hyperparameters
sizes = ['1_tiny', '2_small', '3_standard', '4_full']

size_folders = ['size_data_f1/' + size for size in sizes]

#datasets
datasets = ['sst2']

#number of output classes
num_classes_list = [2, 2, 2, 6, 2]

#number of augmentations per original sentence
n_aug_list_dict = {'size_data_f1/1_tiny': [32, 32, 32, 32, 32], 
					'size_data_f1/2_small': [32, 32, 32, 32, 32],
					'size_data_f1/3_standard': [16, 16, 16, 16, 4],
					'size_data_f1/4_full': [16, 16, 16, 16, 4]}

if not os.path.isdir('size_data_f1'):
    os.mkdir('size_data_f1')
#number of words for input
input_size_list = [50, 50, 40, 25, 25]

#alphas = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
alphas = [0.05, 0.2, 0.4, 0.5]
#alphas = [0.05, 0.15, 0.35, 0.5]

# Number of words for input
input_size_list = [50,50,40,25,25]

#word2vec dictionary
huge_word2vec = 'word2vec/glove.840B.300d.txt'
word2vec_len = 300

In [3]:
def run_cnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_cnn(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=10, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=0)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [4]:
def run_rnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_model(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=10, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=0)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [8]:
#for each method
## TODO: add original EDA
writer_cnn = open('outputs_f1/cnn_general'+'_' + get_now_str() + '.txt', 'w')
writer_rnn = open('outputs_f1/rnn_general' + '_' + get_now_str() + '.txt', 'w')


#for each size dataset
for size_folder in size_folders:

    writer_cnn.write(size_folder + '\n')
    writer_rnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]

    #for storing the performances
    performances_weda = {alpha:[] for alpha in alphas}
    performances_eda = {alpha:[] for alpha in alphas}

    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset_folder + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        #test each alpha value
        for alpha in alphas:

            train_path = dataset_folder + '/train_weda_' + str(alpha) + '.txt'
            test_path = 'size_data_f1/test/' + dataset + '/test.txt'
            acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
            print("cnn weda aug ", acc, alpha, dataset_folder)

            performances_weda[alpha].append(acc)
        
            train_path = dataset_folder + '/train_eda_' + str(alpha) + '.txt'
            acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
            performances_eda[alpha].append(acc)
            print("cnn eda aug", acc)
            

    writer_cnn.write(str(performances_weda) + '\n')
    writer_rnn.write(str(performances_eda)+"\n")
    print("Performances WEDA")
    for alpha in performances_weda:
        line = str(alpha) + ' : ' + str(sum(performances_weda[alpha])/len(performances_weda[alpha]))
        writer_cnn.write(line + '\n')
        print(line)
    
    print("Performances EDA")
    for alpha in performances_eda:
        line = str(alpha) + ' : ' + str(sum(performances_eda[alpha])/len(performances_eda[alpha]))
        writer_rnn.write(line + '\n')
        print(line)
    
    print(performances_weda)
    print(performances_eda)

writer_cnn.close()
writer_rnn.close()
    
## TODO: add training without augmentation

cnn aug  0.740423098913665 0.05 size_data_f1/1_tiny/sst2
cnn aug  0.740423098913665 0.2 size_data_f1/1_tiny/sst2
cnn aug  0.7381360777587193 0.4 size_data_f1/1_tiny/sst2
cnn aug  0.7352773013150372 0.5 size_data_f1/1_tiny/sst2
0.05 : 0.740423098913665
0.2 : 0.740423098913665
0.4 : 0.7381360777587193
0.5 : 0.7352773013150372
{0.05: [0.740423098913665], 0.2: [0.740423098913665], 0.4: [0.7381360777587193], 0.5: [0.7352773013150372]}
{0.05: [], 0.2: [], 0.4: [], 0.5: []}
cnn aug  0.7895940537449971 0.05 size_data_f1/2_small/sst2
cnn aug  0.7981703830760435 0.2 size_data_f1/2_small/sst2
cnn aug  0.7941680960548885 0.4 size_data_f1/2_small/sst2
cnn aug  0.793596340766152 0.5 size_data_f1/2_small/sst2
0.05 : 0.7895940537449971
0.2 : 0.7981703830760435
0.4 : 0.7941680960548885
0.5 : 0.793596340766152
{0.05: [0.7895940537449971], 0.2: [0.7981703830760435], 0.4: [0.7941680960548885], 0.5: [0.793596340766152]}
{0.05: [], 0.2: [], 0.4: [], 0.5: []}
cnn aug  0.8421955403087479 0.05 size_data_f1/3_s

In [9]:

writer_cnn = open('outputs_f1/cnn_no_aug_' + get_now_str() + '.txt', 'w')
writer_rnn = open('outputs_f1/rnn_no_aug_' + get_now_str() + '.txt', 'w')


#for each size dataset
for size_folder in size_folders:

    writer_cnn.write(size_folder + '\n')
    writer_rnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]

    #for storing the performances
    performances_rnn = []
    performances_cnn = []

    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset_folder + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        train_path = dataset_folder + '/train_orig.txt'
        test_path = 'size_data_f1/test/' + dataset + '/test.txt'
        
        acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
        print("orignal no aug", acc)
        performances_cnn.append(acc)

        """
        acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
        performances_rnn.append(acc)
        print("rnn", acc)
        """

    writer_cnn.write(str(performances_cnn) + '\n')
    writer_rnn.write(str(performances_rnn)+"\n")

    print(performances_cnn)
    print(performances_rnn)

writer_cnn.close()
writer_rnn.close()



cnn 0.6815323041738136
[0.6815323041738136]
[]
cnn 0.7821612349914236
[0.7821612349914236]
[]
cnn 0.8279016580903373
[0.8279016580903373]
[]
cnn 0.8376214979988564
[0.8376214979988564]
[]
